1. Downloaded big dataset of cars (1gb) from Kaggle.

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint




In [43]:
# Set the path to the directory containing the training and testing images
train_dir = 'data/cars_train/'
test_dir = 'data/cars_test/'

In [44]:
# Define image preprocessing options
train_datagen = ImageDataGenerator(
    rescale=1./255,   # Rescale pixel values to [0,1]
    rotation_range=20,   # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,   # Randomly shift images horizontally
    height_shift_range=0.2,   # Randomly shift images vertically
    shear_range=0.2,   # Randomly apply shearing transformation
    zoom_range=0.2,   # Randomly zoom in on images
    horizontal_flip=True,   # Randomly flip images horizontally
    fill_mode='nearest'   # Fill in missing pixels with nearest value
)

In [45]:
test_datagen = ImageDataGenerator(rescale=1./255)
img_size = (224, 224)  # set image size
batch_size = 8

In [46]:
# Load the training and testing images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 8144 images belonging to 1 classes.
Found 8041 images belonging to 1 classes.


In [47]:
# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

In [48]:
# Add a new classification layer on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [49]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [50]:
# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Define callbacks to save the best model and stop training early if no improvement
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [51]:
# Compile the model with appropriate optimizer and loss function
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [52]:
# Train the model
history = model.fit(
    train_generator, 
    epochs=50,
    validation_data=test_generator,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/50
 817/1018 [=======================>......] - ETA: 1:45 - loss: 5.6347e-04 - accuracy: 1.0000

KeyboardInterrupt: 

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0
